<a href="https://colab.research.google.com/github/GarvinAgweli/personal/blob/main/INTROVERT/EXTROVERT%20PREDICTION%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report,roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import  precision_recall_curve, average_precision_score
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import xgboost as xgb
import statsmodels.api as sm
import pickle
import gc
from sklearn import svm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
df=pd.read_csv('sample_submission.csv')

In [ ]:
df.head()

,id,Personality
0,18524,Extrovert
1,18525,Extrovert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Extrovert


In [ ]:
df.shape

(6175, 2)

In [ ]:
df.columns

Index(['id', 'Personality'], dtype='object')

In [ ]:
df.describe()

,id
count,6175.000000
mean,21611.000000
std,1782.713288
min,18524.000000
25%,20067.500000
50%,21611.000000
75%,23154.500000
max,24698.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6175 entries, 0 to 6174
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           6175 non-null   int64 
 1   Personality  6175 non-null   object
dtypes: int64(1), object(1)
memory usage: 96.6+ KB


In [ ]:
# Stimulate features for demo
np.random.seed(42)

# Create a balanced distribution of 'Introvert' and 'Extrovert' for demonstration
num_samples = len(df)
introvert_count = num_samples // 2
extrovert_count = num_samples - introvert_count
personality_list = ['Introvert'] * introvert_count + ['Extrovert'] * extrovert_count
np.random.shuffle(personality_list)
df['Personality'] = personality_list

df['social_events_per_week']=np.where(df['Personality']=='Extrovert',
                                      np.random.randint(4,10,len(df)),
                                      np.random.randint(0,3,len(df)))
df['time_spent_alone_hours']=np.where(df['Personality']=='Introvert',
                                      np.random.randint(30,50,len(df)),
                                      np.random.randint(5,20,len(df)))
df['prefers_texting']=np.where(df['Personality']=='Introvert',1,0)
df['group_activity']=np.where(df['Personality']=='Extrovert',
                                      np.random.randint(10,20,len(df)),
                                      np.random.randint(0,5,len(df)))

In [ ]:
df['target']=df['Personality'].map({'Introvert':1,'Extrovert':0})

In [ ]:
X=df[['social_events_per_week','time_spent_alone_hours','prefers_texting','group_activity']]
y=df[['target']]

In [ ]:
imputer = IterativeImputer(max_iter=50, random_state=0)
imputed_data = imputer.fit_transform(X)
imp_original = pd.DataFrame(imputed_data, columns=X.columns)
display(imp_original.head())

,social_events_per_week,time_spent_alone_hours,prefers_texting,group_activity
0,1.0,49.0,1.0,3.0
1,1.0,48.0,1.0,4.0
2,4.0,12.0,0.0,16.0
3,1.0,49.0,1.0,2.0
4,4.0,5.0,0.0,12.0


In [ ]:
imputer = IterativeImputer(max_iter=50, random_state=0)
imputed_data = imputer.fit_transform(X_train)
imp_train = pd.DataFrame(imputed_data, columns=X_train.columns)
display(imp_original.head())

,social_events_per_week,time_spent_alone_hours,prefers_texting,group_activity
0,1.0,49.0,1.0,3.0
1,1.0,48.0,1.0,4.0
2,4.0,12.0,0.0,16.0
3,1.0,49.0,1.0,2.0
4,4.0,5.0,0.0,12.0


In [ ]:
imputer = IterativeImputer(max_iter=50, random_state=0)
imputed_data = imputer.fit_transform(X_test)
imp_test = pd.DataFrame(imputed_data, columns=X_test.columns)
display(imp_original.head())

,social_events_per_week,time_spent_alone_hours,prefers_texting,group_activity
0,1.0,49.0,1.0,3.0
1,1.0,48.0,1.0,4.0
2,4.0,12.0,0.0,16.0
3,1.0,49.0,1.0,2.0
4,4.0,5.0,0.0,12.0


In [ ]:
fin_train = pd.concat([imp_train, imp_original, imp_original, imp_original, imp_original], axis = 0)

In [ ]:
X = imp_train
y = df.loc[X.index, 'target']

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train.head()

,social_events_per_week,time_spent_alone_hours,prefers_texting,group_activity
375,4.0,13.0,0.0,17.0
673,6.0,11.0,0.0,17.0
350,4.0,18.0,0.0,10.0
299,0.0,43.0,1.0,4.0
18,8.0,13.0,0.0,19.0


In [ ]:
y_train.head()

,target
375,0
673,1
350,1
299,0
18,0


In [ ]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
rf.score(X_train,y_train)

0.8886894075403949

In [ ]:
rf.score(X_test,y_test)

0.4690909090909091

In [ ]:
y_pred=rf.predict(X_test)
y_pred_proba=rf.predict_proba(X_test)
print('Classification report:\n', classification_report(y_test, y_pred))
print('Confusion matrix:\n',confusion_matrix(y_true = y_test, y_pred = y_pred))
print("Logistic Regression Accuracy: ",accuracy_score(y_test,y_pred))
print('ROC AUC : ', roc_auc_score(y_test, y_pred))

Classification report:
               precision    recall  f1-score   support

           0       0.40      0.44      0.42       121
           1       0.53      0.49      0.51       154

    accuracy                           0.47       275
   macro avg       0.47      0.47      0.47       275
weighted avg       0.47      0.47      0.47       275

Confusion matrix:
 [[53 68]
 [78 76]]
Logistic Regression Accuracy:  0.4690909090909091
ROC AUC :  0.4657615112160567


In [ ]:
lr=LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression()

In [ ]:
lr.score(X_train,y_train)

0.5314183123877917

In [ ]:
lr.score(X_test,y_test)

0.4727272727272727

In [ ]:
y_pred=lr.predict(X_test)
y_pred_proba=lr.predict_proba(X_test)
print('Classification report:\n', classification_report(y_test, y_pred))
print('Confusion matrix:\n',confusion_matrix(y_true = y_test, y_pred = y_pred))
print("Logistic Regression Accuracy: ",accuracy_score(y_test,y_pred))
print('ROC AUC : ', roc_auc_score(y_test, y_pred))


Classification report:
               precision    recall  f1-score   support

           0       0.42      0.55      0.48       121
           1       0.54      0.41      0.46       154

    accuracy                           0.47       275
   macro avg       0.48      0.48      0.47       275
weighted avg       0.49      0.47      0.47       275

Confusion matrix:
 [[67 54]
 [91 63]]
Logistic Regression Accuracy:  0.4727272727272727
ROC AUC :  0.4814049586776859


In [ ]:
pred = lr.predict(imp_test)

In [ ]:
sam_lr=df.copy()

In [ ]:
sam_lr.head()

,id,Personality,social_events_per_week,time_spent_alone_hours,prefers_texting,group_activity,target
0,18524,Introvert,1,49,1,3,1
1,18525,Introvert,1,48,1,4,1
2,18526,Extrovert,4,12,0,16,0
3,18527,Introvert,1,49,1,2,1
4,18528,Extrovert,4,5,0,12,0


In [ ]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
y_pred=xgb.predict(X_test)
y_Pred_proba=xgb.predict_proba(X_test)
print('\nClassification Report',classification_report(y_test,y_pred))
print('\nConfusion Matrix',confusion_matrix(y_test,y_pred))
print(f'ROC AUC Score::',roc_auc_score(y_test,y_pred))


Classification Report               precision    recall  f1-score   support

           0       0.41      0.43      0.42       121
           1       0.53      0.51      0.52       154

    accuracy                           0.48       275
   macro avg       0.47      0.47      0.47       275
weighted avg       0.48      0.48      0.48       275


Confusion Matrix [[52 69]
 [75 79]]
ROC AUC Score:: 0.47136953955135774


In [ ]:
xgb.score(X_train,y_train)

0.8563734290843806

In [ ]:
xgb.score(X_test,y_test)

0.4763636363636364

In [ ]:
pred = xgb.predict(X_test)
sam_xgb_pred = pd.DataFrame({'Personality': pred}, index=X_test.index)
sam_xgb = df.copy()
sam_xgb.loc[sam_xgb_pred.index, 'Personality'] = sam_xgb_pred['Personality']
sam_xgb['Personality'] = sam_xgb['Personality'].map({0: 'Extrovert', 1 : 'Introvert'})


In [ ]:
sam_xgb=df.copy()

In [ ]:
sam_xgb.head()

,id,Personality,social_events_per_week,time_spent_alone_hours,prefers_texting,group_activity,target
0,18524,Introvert,1,49,1,3,1
1,18525,Introvert,1,48,1,4,1
2,18526,Extrovert,4,12,0,16,0
3,18527,Introvert,1,49,1,2,1
4,18528,Extrovert,4,5,0,12,0
